In [0]:
%load_ext autoreload
%autoreload 1

In [0]:
!pip install tensorflow-gpu==2.0.0-alpha0

In [0]:
import os,sys,inspect
import os
import joblib
import tensorflow as tf
import numpy as np
import h5py
import scipy.sparse.linalg as la
import scipy.sparse as sp
import scipy
import time
import pickle

import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Layer, Dense, Flatten, Activation, Dropout, LeakyReLU
from tensorflow.keras.regularizers import l2
from tensorflow.keras import Model

import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
%matplotlib inline

import scipy.io as sio

In [0]:
%aimport graph
%aimport coarsening
%aimport utils

In [0]:
# Graphs.
number_edges = 8
metric ='euclidean'
normalized_laplacian = True
coarsening_levels = 4
len_img = 28

np.random.seed(0)

In [0]:
# Useful functions

def grid_graph(m):
  z = graph.grid(m)  # normalized nodes coordinates
  dist, idx = graph.distance_sklearn_metrics(z, k=number_edges, metric=metric) 
  #dist contains the distance of the 8 nearest neighbors for each node indicated in z sorted in ascending order
  #idx contains the indexes of the 8 nearest for each node sorted in ascending order by distance

  A = graph.adjacency(dist, idx)  # graph.adjacency() builds a sparse matrix out of the identified edges computing similarities as: A_{ij} = e^(-dist_{ij}^2/sigma^2)

  return A, z

  
def plot_matrix(m):
  plt.figure(figsize = (10, 10))
  plt.imshow(m.toarray())
  plt.show


# A friend helped me a lot with this function, so you will find something similar in other assignment
def coarsen_mnist(A, levels, nodes_coordinates):
    graphs, parents = coarsening.metis(A, levels) #Coarsen a graph multiple times using Graclus variation of the METIS algorithm. 
                                                  #Basically, we randomly sort the nodes, we iterate on them and we decided to group each node
                                                  #with the neighbor having highest w_ij * 1/(\sum_k w_ik) + w_ij * 1/(\sum_k w_kj) 
                                                  #i.e. highest sum of probabilities to randomly walk from i to j and from j to i.
                                                  #We thus favour strong connections (i.e. the ones with high weight wrt all the others for both nodes) 
                                                  #in the choice of the neighbor of each node.
                    
                                                  #Construction is done a priori, so we have one graph for all the samples!
                    
                                                  #graphs = list of spare adjacency matrices (it contains in position 
                                                  #          0 the original graph)
                                                  #parents = list of numpy arrays (every array in position i contains 
                                                  #           the mapping from graph i to graph i+1, i.e. the idx of
                                                  #           node i in the coarsed graph -> that is, the idx of its cluster) 
    perms = coarsening.compute_perm(parents) #Return a list of indices to reorder the adjacency and data matrices so
                                             #that two consecutive nodes correspond to neighbors that should be collapsed
                                             #to produce the coarsed version of the graph.
                                             #Fake nodes are appended for each node which is not grouped with anybody else
    
    coordinates = np.copy(nodes_coordinates)
    u_shape, u_rows, u_cols, u_val = [], [], [], []
    
    for i,A in enumerate(graphs):
        M, M = A.shape

        # We remove self-connections created by metis.
        A = A.tocoo()
        A.setdiag(0)

        if i < levels: #if we have to pool the graph 
            A = coarsening.perm_adjacency(A, perms[i]) #matrix A is here extended with the fakes nodes
                                                       #in order to do an efficient pooling operation
                                                       #in tensorflow as it was a 1D pooling

        A = A.tocsr()
        A.eliminate_zeros()
        
        Mnew, Mnew = A.shape
        u_shape.append(Mnew)
        
        if i == 0:
            fake_nodes = Mnew - M
            coordinates = np.concatenate([coordinates, np.ones([fake_nodes, 2])*np.inf], 0)
            coordinates = coordinates[perms[0]]
        
        start_node, end_node = A.nonzero()
        u_rows.append(start_node)
        u_cols.append(end_node)
        
        distance = coordinates[start_node] - coordinates[end_node]
        u_val.append(distance)
        
        print('Layer {0}: M_{0} = |V| = {1} nodes ({2} added), |E| = {3} edges'.format(i, Mnew, Mnew-M, A.nnz//2))
        
        # update coordinates for next coarser graph
        new_coordinates = []
        for k in range(A.shape[0]//2):
            idx_first_el = k * 2
            
            if not np.isfinite(coordinates[idx_first_el][0]):
                new_coordinates.append(coordinates[idx_first_el+1])
                
            elif not np.isfinite(coordinates[idx_first_el+1][0]):
                new_coordinates.append(coordinates[idx_first_el])
                
            else:
                new_coordinates.append(np.mean(coordinates[idx_first_el:idx_first_el+2], axis=0))
                
        coordinates = np.asarray(new_coordinates)

    return u_shape, u_rows, u_cols, u_val, perms[0]


In [237]:
# Load MNIST dataset

mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype(np.float32) / 255
x_test = x_test.astype(np.float32) / 255
y_train = y_train.astype(np.float32) 
y_test = y_test.astype(np.float32)

val_n = len(x_train)//100 * 15
(x_val, y_val) = x_train[0:val_n], y_train[0:val_n]
(x_train, y_train) = x_train[val_n:], y_train[val_n:]
x_val = x_val.astype(np.float32)
y_val = y_val.astype(np.float32)

x_train_original = x_train.reshape(-1, len_img*len_img)
x_val_original = x_val.reshape(-1, len_img*len_img)
x_test_original = x_test.reshape(-1, len_img*len_img)

print(x_train_original.shape)

(51000, 784)


In [238]:
# Create u

n_rows_cols = 28
A, nodes_coordinates = grid_graph(n_rows_cols)
u_shape, u_rows, u_cols, u_val, perm = coarsen_mnist(A, coarsening_levels, nodes_coordinates)

u = []
for level in range(coarsening_levels):
    u.append([u_shape[level], u_rows[level], u_cols[level], u_val[level]])

Layer 0: M_0 = |V| = 992 nodes (208 added), |E| = 3198 edges
Layer 1: M_1 = |V| = 496 nodes (89 added), |E| = 1425 edges
Layer 2: M_2 = |V| = 248 nodes (37 added), |E| = 647 edges
Layer 3: M_3 = |V| = 124 nodes (13 added), |E| = 306 edges
Layer 4: M_4 = |V| = 62 nodes (0 added), |E| = 159 edges


In [239]:
# Coarsen dataset

x_train = coarsening.perm_data(x_train_original, perm)
x_val = coarsening.perm_data(x_val_original, perm)
x_test = coarsening.perm_data(x_test_original, perm)

x_train = np.expand_dims(x_train, -1)
x_val = np.expand_dims(x_val, -1)
x_test = np.expand_dims(x_test, -1)

print(x_train.shape)

(51000, 992, 1)


In [0]:
# Create Gaussian weightning 

class Weighting(Layer):

  def __init__(self, u, d, batch_size, **kwargs):
    self.u = u
    self.d = d
    self.batch_size = batch_size
    super(Weighting, self).__init__(**kwargs)

  def build(self, input_shape):
    self.mu = self.add_weight(name='mu',
                              shape=(1, self.d),
                              initializer='uniform',
                              trainable=True)
    self.sigma = self.add_weight(name='sigma',
                                 shape=(1, self.d),
                                 initializer='uniform',
                                 trainable=True)
    super(Weighting, self).build(input_shape)  

  def call(self, X):
    batch_size, n_nodes, n_features = X.shape
    batch_size = self.batch_size
    u_shape, u_rows, u_cols, u_val = self.u

    diff =  tf.square(u_val - self.mu)
    factor = tf.square(self.sigma) + 1e-14
    weights = -0.5 * (diff / factor)
    weights = tf.math.reduce_sum(weights, axis=1)

    weights = tf.SparseTensor(indices=np.vstack([u_rows, u_cols]).T, 
                              values=weights, 
                              dense_shape=[u_shape]*2)
    weights = tf.sparse.reorder(weights)
    weights = tf.sparse.softmax(weights)
    
    X_t = tf.reshape(tf.transpose(X, [1,2,0]), [n_nodes, batch_size * n_features])
    
    D = tf.sparse.sparse_dense_matmul(weights, X_t)
    D = tf.transpose(tf.reshape(D, [n_nodes, n_features, batch_size]), [2,0,1])
    
    return D
    
  def compute_output_shape(self, input_shape):
    return (input_shape[0], self.output_dim)

  
# Create Gaussian (kernel) layer

class MoNet(Layer):

  def __init__(self, u, d, n_gaussian, n_hidden, batch_size, **kwargs):
    self.u = u
    self.d = d
    self.n_gaussian = n_gaussian
    self.n_hidden = n_hidden
    self.batch_size = batch_size
    
    self.weightings = []
    for k in range(self.n_gaussian):
      weighting = Weighting(self.u, self.d, self.batch_size, trainable=True)
      self.weightings.append(weighting)
      
    super(MoNet, self).__init__(**kwargs)

  def build(self, input_shape):
    self.W = self.add_weight(name='W', 
                             shape=(input_shape[-1] * self.n_gaussian, self.n_hidden),
                             initializer='uniform',
                             trainable=True)

    super(MoNet, self).build(input_shape)  # Be sure to call this at the end

  def call(self, X):
    weightings = []
    for k in range(self.n_gaussian):
      weighting = self.weightings[k](X)
      weightings.append(weighting)
    
    weights = tf.transpose(tf.stack(weightings), [1,2,3,0])
    
    batch_size, n_nodes, n_features = X.shape
    batch_size = self.batch_size
    weights = tf.reshape(weights, [batch_size*n_nodes, n_features*self.n_gaussian])
    
    h = weights @ self.W
    h = tf.reshape(h, [batch_size, n_nodes, self.n_hidden])

    return h

  def compute_output_shape(self, input_shape):
    return (input_shape[1], self.n_hidden)

  
class MPool1(Layer):
      def __init__(self, p, **kwargs):
         self.p = p
         super(MPool1, self).__init__(**kwargs)
     
      def call(self, X):
        """Max pooling of size p. Should be a power of 2 (this is possible thanks to the reordering we previously did)."""
        if self.p > 1:
          X = tf.expand_dims(X, 3)  # shape = N x M x F x 1
          X = tf.nn.max_pool(X, ksize=[1,self.p,1,1], strides=[1, self.p,1,1], padding='SAME')
          X = tf.squeeze(X, [3])  # shape = N x M/p x F
        return X
          
      def compute_output_shape(self, input_shape):
        return input_shape

In [0]:
# Hyperparameters

epochs=10
# For this implementation, batch_size needs to be a divisor of the number of samples (x_train = 51000 samples)
batch_size = 50
d = 2
n_classes=10
n_gaussian = 25

dropout = 0.5
learning_rate = 1e-4
reg = l2(5e-4)
momentum = 0.8
p = 4

In [0]:
# Build the model

model = Sequential()
model.add(MoNet(u=u[0], d=d, n_gaussian=n_gaussian, n_hidden=32, batch_size=batch_size, trainable=True))
model.add(LeakyReLU())
model.add(MPool1(p=p))
model.add(MoNet(u=u[2], d=d, n_gaussian=n_gaussian, n_hidden=64, batch_size=batch_size, trainable=True))
model.add(LeakyReLU())
model.add(MPool1(p=p))
model.add(Flatten())
model.add(Dense(512, activation='relu', kernel_regularizer=reg))
model.add(Dropout(dropout))
model.add(Dense(n_classes, activation='softmax', kernel_regularizer=reg))

model.compile(optimizer=tf.optimizers.RMSprop(learning_rate=learning_rate, momentum=momentum),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [252]:
# Train the model

model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=epochs, batch_size=batch_size)

Train on 51000 samples, validate on 9000 samples
Epoch 1/10
51000/51000 [==============================] - 108s 2ms/sample - loss: 0.4449 - accuracy: 0.9330 - val_loss: 0.1792 - val_accuracy: 0.9792
Epoch 2/10
51000/51000 [==============================] - 101s 2ms/sample - loss: 0.1547 - accuracy: 0.9778 - val_loss: 0.1301 - val_accuracy: 0.9816
Epoch 3/10
51000/51000 [==============================] - 101s 2ms/sample - loss: 0.1210 - accuracy: 0.9801 - val_loss: 0.1068 - val_accuracy: 0.9841
Epoch 4/10
51000/51000 [==============================] - 103s 2ms/sample - loss: 0.1094 - accuracy: 0.9822 - val_loss: 0.1082 - val_accuracy: 0.9828
Epoch 5/10
51000/51000 [==============================] - 100s 2ms/sample - loss: 0.1047 - accuracy: 0.9827 - val_loss: 0.0982 - val_accuracy: 0.9850
Epoch 6/10
51000/51000 [==============================] - 100s 2ms/sample - loss: 0.0994 - accuracy: 0.9844 - val_loss: 0.1000 - val_accuracy: 0.9839
Epoch 7/10
51000/51000 [===========================

In [253]:
model.evaluate(x_test, y_test, batch_size=batch_size)

10000/10000 [==============================] - 9s 866us/sample - loss: 0.0883 - accuracy: 0.9865


[0.08828231139108539, 0.9865]

In [0]:
# DEBUGGING

# k = MoNet(u=u[0], d=d, n_gaussian=25, n_hidden=32, batch_size=batch_size, trainable=True)
# k(x_train[0:batch_size,:,:])